# zad 1 Jakub Iliński

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Tuple

PAPUGA = 'flax-community/papuGaPT2'
DEVICE = "cuda"

tokenizer = AutoTokenizer.from_pretrained(PAPUGA)
model = AutoModelForCausalLM.from_pretrained(PAPUGA).to(DEVICE)
model.device

/home/hinski2/miniconda3/envs/ml/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda', index=0)

In [ ]:
class Chat:
    def __init__(self) -> None:
        self.chat_history: List[Tuple[str, str]] = []
        self.persona: str = "System: Jesteś asystenetem w sklepie z szeroka gamą produktów wspinaczkowych, twoim zadaniem jest pomóc klientowi w wyborze produktu"
        
    def build_prompt(self, user_input: str) -> str:
        history = self.chat_history[-8:]
        conversation = [f"System: {self.persona}"]
        for role, text in history:
            conversation.append(f"{role}: {text}")
        conversation.append(f"Asystent:")
        return "\n".join(conversation)
        
    def generate_candidates(self, prompt: str, num: int = 100, max_new_tokens: int = 50) -> List[str]:
        input_ids = tokenizer(prompt, return_tensors="pt").to(DEVICE)
        
        out = model.generate(
            **input_ids,
            temperature=0.1,
            top_p=0.95,
            repetition_penalty=1.15,
            no_repeat_ngram_size=3,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            num_return_sequences=num,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )
        
        texts: List[str] = []
        for seq in out:
            full = tokenizer.decode(seq, skip_special_tokens=True)
            new = full[len(prompt):]
            texts.append(new.strip())
            
        return self.handle_text(texts)
    
    def handle_text(self, texts: List[str]) -> List[str]:
        ans: List[str] = []
        for text in texts: 
            dot_pos: int = text.rfind('.')
            
            if dot_pos != -1:
                ans.append(text[:dot_pos + 1])
            else:
                ans.append(text + '.')
                
        return ans
    
    def select_output(self, candidates: List[str]) -> str:
        
        def heuristic(candidate: str) -> int:
            score = 0
            for char in candidate.lower():
                if  ord('a') <= ord(char) <= ord('z') or \
                    ord('0') <= ord(char) <= ord('9') or \
                    char in [' ', ',', '.', '!', '?', 'ń', 'ż', 'ć', 'ź', 'ś', 'ó']:
                    score += 2 
                else:
                    score -= 20
                    
            return score
        
        return min(candidates, key=heuristic)     
    
    def run(self) -> None:
        while True:
            user_input: str = input().strip()
            prompt: str = self.build_prompt(user_input)
            candidates: List[str] = self.generate_candidates(prompt)
            output = self.select_output(candidates)
            print(output)
            
            self.chat_history.append(("Klient", user_input))
            self.chat_history.append(("Asystent", output))

In [ ]:
chat = Chat()
chat.run()

Asystentka sklepu internetowego. Osoba odpowiedzialna za obsługę klienta i sprzedaż na każdym etapie współpracy (od momentu złożenia
osoba zajmująca się obsługą klientów sklepów internetowych oraz ich sprzedażą internetową; odpowiadająca również m/innymi do
pracownik działu obsługi Klienta Sklepu Internetowego lub jego oddziału - odpowiedzialny także np.: przy obsłudze zamówień składanych przez Klientów
sprzedawca-sprzedawca Sklepów Internetowych , który zajmuje sie obsługa sprzedaży internetowej . Odpowiadał też wcześniej
